In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def value_evaluation(s, v_pi, pi_s, grid_size):
    temp = 0.0

    ## move left
    if (s % 4) > 0  : temp += pi_s[s, 0] * (v_pi[s - 1] - 1)
    else            : temp += pi_s[s, 0] * (v_pi[s] - 1)
    ## move right
    if ((s + 1) % 4) > 0: temp += pi_s[s, 1] * (v_pi[s + 1] - 1)
    else                : temp += pi_s[s, 1] * (v_pi[s] - 1)
    ## move up
    if s > 3    : temp += pi_s[s, 2] * (v_pi[s - grid_size] - 1)
    else        : temp += pi_s[s, 2] * (v_pi[s] - 1)
    ## move down
    if s < 12    : temp += pi_s[s, 3] * (v_pi[s + grid_size] - 1)
    else        : temp += pi_s[s, 3] * (v_pi[s] - 1)

    v_pi[s] = temp

    return v_pi

def neighbor_values(s, v_pi):
    vals = np.zeros([4], dtype='f')

    ## left
    if (s % 4) > 0  : vals[0] = v_pi[s - 1]
    else            : vals[0] = -np.Infinity
    ## right
    if ((s + 1) % 4) > 0: vals[1] = v_pi[s + 1]
    else                : vals[1] = -np.Infinity
    ## up
    if s > 3    : vals[2] = v_pi[s - 4]
    else        : vals[2] = -np.Infinity
    ## down
    if s < 12   : vals[3] = v_pi[s + 4]
    else        : vals[3] = -np.Infinity

    return vals

def get_policy(pi):
    actions = {
        0: "L",
        1: "R",
        2: "U",
        3: "D"
    }

    policy = np.full(pi.shape[0], ['U'])
    for i in range(policy.shape[0]):
        policy[i] = actions[np.argmax(pi[i])]

    return policy.reshape([4, 4])


In [3]:
## parameters
gamma = 1.0
grid_size = 4
states = int(grid_size ** 2)

## random initial values and arbitrary initial policy
v_pi = np.random.normal(size=[states])
pi_s = np.full([states, 4], 0.25) ## [left, right, up, down]
v_pi[0] = 0.0
v_pi[15] = 0.0

## random so that loop doesn't break in first iteration
old_v_pi = np.random.normal(size=[states]) 


policy_iterations = 0
epsilon = 1e-12
policy_converged = False

while not policy_converged:
    ## bug fix mentioned in 4.4
    if np.all(old_v_pi == v_pi): break

    policy_iterations += 1
    old_pi = np.copy(pi_s)


    ## value updation
    value_converged = False
    while not value_converged:
        old_v_pi = np.copy(v_pi)
        for s in range(1, 15):  
            v_pi = value_evaluation(s, v_pi, pi_s, grid_size)

        if (np.all(abs(old_v_pi - v_pi) <= epsilon)):
            value_converged = True

    ## policy updation
    for s in range(1, 15):
        pi_s[s] = np.zeros([4])
        greedy_action = np.argmax(neighbor_values(s, v_pi))
        pi_s[s][greedy_action] = 1.0

    ## convergence
    if np.all(abs(old_pi - pi_s) <= epsilon):
        policy_converged = True

    ## print value and policy
    print("\n\npolicy iteration:", policy_iterations)
    print("Values:")
    print(v_pi.reshape([4, 4]))
    print("Greedy policy")
    print(get_policy(pi_s))

print("\n\nFinal Iteration")
print("Values:")
print(v_pi.reshape([4, 4]))
print("Greedy policy")
print(get_policy(pi_s))
print("\n\nTotal policy iterations taken for convergence:", policy_iterations)



policy iteration: 1
Values:
[[  0. -14. -20. -22.]
 [-14. -18. -20. -20.]
 [-20. -20. -18. -14.]
 [-22. -20. -14.   0.]]
Greedy policy
[['L' 'L' 'L' 'L']
 ['U' 'L' 'L' 'D']
 ['U' 'R' 'R' 'D']
 ['R' 'R' 'R' 'L']]


policy iteration: 2
Values:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]
Greedy policy
[['L' 'L' 'L' 'L']
 ['U' 'L' 'L' 'D']
 ['U' 'L' 'R' 'D']
 ['R' 'R' 'R' 'L']]


Final Iteration
Values:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]
Greedy policy
[['L' 'L' 'L' 'L']
 ['U' 'L' 'L' 'D']
 ['U' 'L' 'R' 'D']
 ['R' 'R' 'R' 'L']]


Total policy iterations taken for convergence: 2
